In [1]:
%load_ext sql

In [2]:
%sql sqlite:///publications.db

'Connected: @publications.db'

In [3]:
%%sql

select tbl_name
from sqlite_master
where type = "table"

 * sqlite:///publications.db
Done.


tbl_name
authors
discounts
employee
jobs
pub_info
publishers
roysched
sales
stores
titleauthor


### Challenge 1 - Most Profiting Authors

#### Step 1: Calculate the royalties of each sales for each author

In [4]:
%%sql
select t.title_id as TITLE_ID, a.au_id as AUTHOR_ID,round((t.price * s.qty * t.royalty/100 * ta.royaltyper/100),2) as SALES_ROYALTY  
from authors a 
inner join titleauthor ta on a.au_id = ta.au_id
inner join titles t on ta.title_id = t.title_id
inner join sales s on t.title_id = s.title_id
order by AUTHOR_ID

 * sqlite:///publications.db
Done.


TITLE_ID,AUTHOR_ID,SALES_ROYALTY
PS3333,172-32-1176,29.99
BU1032,213-46-8915,4.0
BU1032,213-46-8915,8.0
BU2075,213-46-8915,25.12
PC1035,238-95-7766,110.16
BU1111,267-41-2394,11.95
TC7777,267-41-2394,8.99
BU7832,274-80-9391,29.99
BU1032,409-56-7008,6.0
BU1032,409-56-7008,11.99


### Aggregate the total royalties for each title for each author

In [5]:
%%sql
select t.title_id as TITLE_ID, a.au_id as AUTHOR_ID, round(sum(t.price * s.qty * t.royalty/100 * ta.royaltyper/100),2) as Aggregated_royalties  
from authors a 
inner join titleauthor ta on a.au_id = ta.au_id
inner join titles t on ta.title_id = t.title_id
inner join sales s on t.title_id = s.title_id
group by a.au_id, t.title_id

 * sqlite:///publications.db
Done.


TITLE_ID,AUTHOR_ID,Aggregated_royalties
PS3333,172-32-1176,29.99
BU1032,213-46-8915,11.99
BU2075,213-46-8915,25.12
PC1035,238-95-7766,110.16
BU1111,267-41-2394,11.95
TC7777,267-41-2394,8.99
BU7832,274-80-9391,29.99
BU1032,409-56-7008,17.99
PC8888,427-17-2319,50.0
TC7777,472-27-2349,8.99


### Step 3: Calculate the total profits of each author

In [12]:
%%sql
with tabla as (select t.title_id as TITLE_ID, a.au_id as AUTHOR_ID, t.advance + round(sum(t.price * s.qty * t.royalty/100 * ta.royaltyper/100),2) as profits  
from authors a 
inner join titleauthor ta on a.au_id = ta.au_id
inner join titles t on ta.title_id = t.title_id
inner join sales s on t.title_id = s.title_id
group by a.au_id, t.title_id
order by author_id)
select AUTHOR_ID, round(sum(profits),2) as PROFITS
from tabla
group by AUTHOR_ID
order by PROFITS desc
limit 3

 * sqlite:///publications.db
Done.


AUTHOR_ID,PROFITS
899-46-2035,17353.14
213-46-8915,15162.11
722-51-5454,15021.53


### Challenge 2

In [9]:
%%sql
create table most_profiting_authors as
with tabla as (select t.title_id as TITLE_ID, a.au_id as AUTHOR_ID, t.advance + round(sum(t.price * s.qty * t.royalty/100 * ta.royaltyper/100),2) as profits  
from authors a 
inner join titleauthor ta on a.au_id = ta.au_id
inner join titles t on ta.title_id = t.title_id
inner join sales s on t.title_id = s.title_id
group by a.au_id, t.title_id
order by author_id)
select AUTHOR_ID, round(sum(profits),2) as PROFITS
from tabla
group by AUTHOR_ID
order by PROFITS desc
limit 3

 * sqlite:///publications.db
Done.


[]

In [10]:
%%sql
select*
from most_profiting_authors

 * sqlite:///publications.db
Done.


AUTHOR_ID,PROFITS
899-46-2035,17353.14
213-46-8915,15162.11
722-51-5454,15021.53
